In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Lecture de tous les mots
words = open('dataset/names.txt', 'r').read().splitlines()
words [:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
#Création des dictionnaires
chars = sorted(list(set(''.join(words)))) # estraction de chaque caracère de la liste de mots
stoi = {s:i+1 for i,s in enumerate(chars)} # Création d'un liste en commençant à l'indice 1 ou chaque caractère est associcé à un index
stoi['.'] = 0 #jout du caractère "." à l'indice 0
itos = {i:s for s,i in stoi.items()} # Création d'une liste inverse pour avoir le nombre et la corespondance du caractere
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [4]:
#Construction du dataset
block_size = 3

X, Y = [], [] #Création de deux liste X pour les entrées et Y pour la sortie attendue

for w in words [:5]:
    print (w)
    context = [0] * block_size # On initie le contexte avec [0, 0, 0]
    for ch in w + '.': # Pour chauqe catactère du mot

        ix = stoi[ch] # On récupère la valeur du nombre correspondant à la lettre
        X.append(context) # On ajoute le contexte à la liste X
        Y.append(ix) # On ajoute la valeur numérique du caractère à la liste Y
        
        #On affiche le contexte + ----> + le caractère du mot que l'on traite 
        print(''.join(itos[i] for i in context), '--->', itos[ix])
        # La même chose mais on affiche les indexes
        #print(''.join(str(i) for i in context), '--->', ix)

        
        
        # On jette le premier élément du contexte on d&calle tout a gauche
        #et on ajoute la valeur numérique de la lettre dans le contexte
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)
        

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [6]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [14]:
#Création de la matrice des embeddings, ou chaque caractère est 
# représenté par une matrice a deux dimensions
C = torch.randn((27, 2))
#print (C)

In [ ]:
######################################################
####### CONSTRUCTION DE LA COUCHE D'EMBEDDING ########
######################################################

In [39]:
# Embedding de la matric X
# A ce stade X contient les contexte de 3 index
# Le réseau ne peut pas travailler avec ces indexes c'est pourquoi on va 
# embedder X pour donner un représentation mathématique des caractères via
# des vecteurs denses
# Avant embedding = X[i] = [5, 13, 13]
# Après embedding [[1.7, -0.3], [0.2, +1.5], [0.2, +1.5]]
# [
      #C[5],    # embedding de 'e' → [1.7, -0.3]
      #C[13],   # embedding de 'm' → [0.2, +1.5]
      #C[13]    # embedding de 'm' → [0.2, +1.5]
# ]
emb = C[X]
# On obtient un tenseur de 32 lignes, dans chaque ligne 3 éléments (contexte) 
# et pour chaque élément la version embedder donc une matrice de 2 dimensions
emb.shape

torch.Size([32, 3, 2])

In [40]:
######################################################
######### CONSTRUCTION DE LA HIDDEN LAYER ############
######################################################

In [51]:
#Initialisatin des poids aléatoirement
W1 = torch.randn((6, 100)) 
# 6 lignes car 
# 6 entrées car notre embedding contient 6 valeurs par ligne 
# 3 lettres de contexte * 2 embedding par lettre  
# [[1.7, -0.3], [0.2, +1.5], [0.2, +1.5]]
# La matrice est applatit avant d'être envoyé au neuronne pour ne faire q'un seul vecteur
# [1.7, -0.3, 0.2, 1.5, 0.2, 1.5]   # vecteur de taille 6

# 100 neuronnes cachés avec 6 poids

#Initialisation des biais aléatoirement
b1 = torch.randn(100)
# 100 biais car 100 neuronnes
#Chaque neuronne reçoit la valeur de l'embedding applati,


# Il fait son calcul tanh(aX + b) et en sortie on a une valeur
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

#Explication du emb.view
# A ce stade on veut muliplier une matrice emb de dimensions [32, 3, 2]
# par la matrice W1 de dimension [6, 100]
# Ce qui est imossible, la multiplication d'une matrice A par une matrice B impose
# que le nombre de colonne de la matrice A soit égal au nombre de ligne de la matrice B
# Il faut donc que la matrice emb ai 6 colonne
# la fonction view permet, sans créer de nouveau tenseur en mémoire de transoformer 
# la forme de notre matrice et pour garder un forme dynamique on indique -1 au niveau
# de la ligne, torch sait alors que ce sera forcéement 32 dans notre cas car il fait
# autmatiquement le calcul en fonction de la taille du tenseur d'origine
# origine 32 * 3 * 2 = 192 donc si on indique 6 il fait 192 / 6 pour déduire 32

print('shape emb :', emb.shape)
print('shape W1 :', W1.shape)
print('shape emb.view(-1, 6) :', emb.view(-1, 6).shape)

shape emb : torch.Size([32, 3, 2])
shape W1 : torch.Size([6, 100])
shape emb.view(-1, 6) : torch.Size([32, 6])


In [52]:
######################################################
######## CONSTRUCTION DE LA COUCHE DE SORTIE #########
######################################################

In [ ]:
#Reprendre la vidéo à 29 minutes